# ML CI/CD 파이프라인

이 노트북은 머신러닝 프로젝트의 CI/CD 파이프라인 개념과 자동화를 보여줍니다.

## 학습 내용
- ML 파이프라인 구조
- 자동화된 학습 및 평가
- 모델 테스트 및 검증
- GitHub Actions를 통한 CI/CD

In [ ]:
# 필요한 라이브러리 설치
!pip install -q scikit-learn pandas numpy pyyaml

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import joblib
import yaml
import json
from pathlib import Path
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

## 1. 파이프라인 설정 파일

In [ ]:
# 파이프라인 설정
pipeline_config = {
    'data': {
        'test_size': 0.2,
        'random_state': 42
    },
    'model': {
        'type': 'RandomForestClassifier',
        'params': {
            'n_estimators': 100,
            'max_depth': 10,
            'random_state': 42
        }
    },
    'training': {
        'cross_validation_folds': 5,
        'min_accuracy': 0.90
    },
    'deployment': {
        'model_path': 'models/',
        'model_name': 'wine_classifier.pkl'
    }
}

# 설정 파일 저장
with open('pipeline_config.yaml', 'w') as f:
    yaml.dump(pipeline_config, f, default_flow_style=False)

print("파이프라인 설정:")
print(yaml.dump(pipeline_config, default_flow_style=False))

## 2. 데이터 준비 단계

In [ ]:
def prepare_data(config):
    """
    데이터 로드 및 전처리
    """
    # 데이터 로드
    wine = load_wine()
    X = wine.data
    y = wine.target
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=config['data']['test_size'],
        random_state=config['data']['random_state'],
        stratify=y
    )
    
    # 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("✓ 데이터 준비 완료")
    print(f"  학습 데이터: {X_train_scaled.shape}")
    print(f"  테스트 데이터: {X_test_scaled.shape}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

# 실행
X_train, X_test, y_train, y_test, scaler = prepare_data(pipeline_config)

## 3. 모델 학습 단계

In [ ]:
def train_model(X_train, y_train, config):
    """
    모델 학습 및 교차 검증
    """
    # 모델 생성
    model_params = config['model']['params']
    model = RandomForestClassifier(**model_params)
    
    # 교차 검증
    cv_folds = config['training']['cross_validation_folds']
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds)
    
    print("✓ 교차 검증 완료")
    print(f"  CV Scores: {cv_scores}")
    print(f"  평균: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    
    # 전체 데이터로 최종 학습
    model.fit(X_train, y_train)
    
    print("✓ 모델 학습 완료")
    
    return model, cv_scores

# 실행
model, cv_scores = train_model(X_train, y_train, pipeline_config)

## 4. 모델 평가 단계

In [ ]:
def evaluate_model(model, X_test, y_test, config):
    """
    모델 평가 및 검증
    """
    # 예측
    y_pred = model.predict(X_test)
    
    # 정확도 계산
    accuracy = accuracy_score(y_test, y_pred)
    
    print("✓ 모델 평가 완료")
    print(f"  테스트 정확도: {accuracy:.4f}")
    
    # 최소 정확도 요구사항 확인
    min_accuracy = config['training']['min_accuracy']
    if accuracy >= min_accuracy:
        print(f"  ✓ 정확도 요구사항 충족 (>= {min_accuracy})")
        passed = True
    else:
        print(f"  ✗ 정확도 요구사항 미충족 (< {min_accuracy})")
        passed = False
    
    # 분류 리포트
    print("\n분류 리포트:")
    print(classification_report(y_test, y_pred))
    
    # 메트릭 딕셔너리
    metrics = {
        'test_accuracy': float(accuracy),
        'cv_mean': float(cv_scores.mean()),
        'cv_std': float(cv_scores.std()),
        'passed_validation': passed
    }
    
    return metrics, passed

# 실행
metrics, passed = evaluate_model(model, X_test, y_test, pipeline_config)

## 5. 모델 저장 단계

In [ ]:
def save_model_artifacts(model, scaler, metrics, config):
    """
    모델 및 아티팩트 저장
    """
    # 디렉토리 생성
    model_dir = Path(config['deployment']['model_path'])
    model_dir.mkdir(exist_ok=True)
    
    # 모델 저장
    model_path = model_dir / config['deployment']['model_name']
    joblib.dump(model, model_path)
    print(f"✓ 모델 저장: {model_path}")
    
    # 스케일러 저장
    scaler_path = model_dir / 'scaler.pkl'
    joblib.dump(scaler, scaler_path)
    print(f"✓ 스케일러 저장: {scaler_path}")
    
    # 메트릭 저장
    metrics_path = model_dir / 'metrics.json'
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"✓ 메트릭 저장: {metrics_path}")
    
    # 설정 저장
    config_path = model_dir / 'config.yaml'
    with open(config_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    print(f"✓ 설정 저장: {config_path}")
    
    return str(model_path)

# 실행 (검증 통과한 경우만)
if passed:
    model_path = save_model_artifacts(model, scaler, metrics, pipeline_config)
    print("\n✓ 모델 배포 준비 완료")
else:
    print("\n✗ 모델이 검증 기준을 통과하지 못했습니다. 배포를 중단합니다.")

## 6. 전체 파이프라인 통합

In [ ]:
def run_ml_pipeline(config_path='pipeline_config.yaml'):
    """
    전체 ML 파이프라인 실행
    """
    print("=" * 70)
    print("ML CI/CD 파이프라인 시작")
    print("=" * 70)
    
    # 1. 설정 로드
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print("\n1. 설정 로드 완료")
    
    # 2. 데이터 준비
    print("\n2. 데이터 준비")
    X_train, X_test, y_train, y_test, scaler = prepare_data(config)
    
    # 3. 모델 학습
    print("\n3. 모델 학습")
    model, cv_scores = train_model(X_train, y_train, config)
    
    # 4. 모델 평가
    print("\n4. 모델 평가")
    metrics, passed = evaluate_model(model, X_test, y_test, config)
    
    # 5. 모델 저장
    print("\n5. 모델 저장")
    if passed:
        model_path = save_model_artifacts(model, scaler, metrics, config)
        print("\n" + "=" * 70)
        print("✓ 파이프라인 성공적으로 완료")
        print("=" * 70)
        return True, model_path
    else:
        print("\n" + "=" * 70)
        print("✗ 파이프라인 실패: 모델 검증 미통과")
        print("=" * 70)
        return False, None

# 전체 파이프라인 실행
success, model_path = run_ml_pipeline()

## 7. GitHub Actions 워크플로우 예제

In [ ]:
%%writefile .github_workflows_ml_pipeline.yml
name: ML Pipeline

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]
  schedule:
    - cron: '0 0 * * 0'  # 매주 일요일 실행

jobs:
  train-and-test:
    runs-on: ubuntu-latest
    
    steps:
    - name: Checkout code
      uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.9'
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
    
    - name: Run data validation
      run: |
        python scripts/validate_data.py
    
    - name: Train model
      run: |
        python scripts/train.py
    
    - name: Test model
      run: |
        python scripts/test_model.py
    
    - name: Upload model artifacts
      uses: actions/upload-artifact@v3
      with:
        name: model-artifacts
        path: models/
    
    - name: Check model performance
      run: |
        python scripts/check_performance.py
  
  deploy:
    needs: train-and-test
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    
    steps:
    - name: Checkout code
      uses: actions/checkout@v3
    
    - name: Download model artifacts
      uses: actions/download-artifact@v3
      with:
        name: model-artifacts
        path: models/
    
    - name: Deploy to production
      run: |
        echo "Deploying model to production..."
        # 실제 배포 스크립트 실행
    
    - name: Send notification
      run: |
        echo "Model deployed successfully!"

## 8. 모델 테스트 스크립트

In [ ]:
%%writefile test_model.py
"""모델 단위 테스트"""
import joblib
import numpy as np
from pathlib import Path

def test_model_exists():
    """모델 파일 존재 확인"""
    model_path = Path('models/wine_classifier.pkl')
    assert model_path.exists(), "Model file not found"
    print("✓ Model file exists")

def test_model_prediction():
    """모델 예측 기능 테스트"""
    model = joblib.load('models/wine_classifier.pkl')
    scaler = joblib.load('models/scaler.pkl')
    
    # 샘플 데이터
    sample = np.array([[13.0, 2.0, 2.3, 17.0, 100.0, 2.8, 3.0, 0.3, 2.0, 5.5, 1.0, 3.0, 1200.0]])
    sample_scaled = scaler.transform(sample)
    
    # 예측
    prediction = model.predict(sample_scaled)
    
    assert prediction is not None, "Prediction failed"
    assert len(prediction) == 1, "Invalid prediction shape"
    assert 0 <= prediction[0] <= 2, "Invalid prediction class"
    print("✓ Model prediction works")

def test_model_probability():
    """모델 확률 예측 테스트"""
    model = joblib.load('models/wine_classifier.pkl')
    scaler = joblib.load('models/scaler.pkl')
    
    sample = np.array([[13.0, 2.0, 2.3, 17.0, 100.0, 2.8, 3.0, 0.3, 2.0, 5.5, 1.0, 3.0, 1200.0]])
    sample_scaled = scaler.transform(sample)
    
    probabilities = model.predict_proba(sample_scaled)
    
    assert probabilities is not None, "Probability prediction failed"
    assert abs(probabilities.sum() - 1.0) < 0.01, "Probabilities don't sum to 1"
    print("✓ Model probability prediction works")

if __name__ == '__main__':
    print("Running model tests...")
    test_model_exists()
    test_model_prediction()
    test_model_probability()
    print("\n✓ All tests passed!")

In [ ]:
# 테스트 실행
if success:
    !python test_model.py

## 요약

이 노트북에서는 ML CI/CD 파이프라인에 대해 학습했습니다:
1. ✅ 파이프라인 설정 파일 (YAML)
2. ✅ 데이터 준비 자동화
3. ✅ 모델 학습 및 교차 검증
4. ✅ 모델 평가 및 검증
5. ✅ 모델 아티팩트 저장
6. ✅ 통합 파이프라인 실행
7. ✅ GitHub Actions 워크플로우 예제
8. ✅ 자동화된 모델 테스트

**CI/CD 모범 사례:**
- 버전 관리된 설정 파일 사용
- 자동화된 테스트 및 검증
- 성능 기준 설정 및 검증
- 아티팩트 버전 관리
- 자동 배포 파이프라인
- 롤백 전략 마련

이것으로 MLOps 기본 과정을 완료했습니다! 🎉